In [1]:
training_data = [
    [54.4, 14.4, 'Kylrum'],
    [45.4, 12.4, 'Kylrum'],
    [89.4, 19.5, 'Klassrum'],
    [57.4, 18.1, 'Lärarrum'],
    [22.4, 8.6, 'Kylrum'],
    [24.4, 11.24, 'Kylrum'],
    [84.4, 24.4, 'Klassrum'],
    [95.4, 22.4, 'Klassrum'],
    [81.4, 20.1, 'Lärarrum'],
    [70, 19.7, 'Lärarrum'],
    [70, 19.8,'Klassrum'],
]

In [2]:
header = ["humidity","temperature","room"]

In [3]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [4]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [5]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [6]:
class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def moreless(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val <= self.value
    
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [7]:
# DEMO, first is attribute to use, second is the compare value
Question(1,3)

Is temperature >= 3?

In [8]:
q = Question(0,60)
example = training_data[0]
q.match(example)

False

In [9]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def partitionMoreLess(rows, question):
    true_rows, false_rows = [],[]
    for row in rows:
        if question.moreless(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [10]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [11]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left))/(len(left)+len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [12]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6611570247933884

In [13]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partitionMoreLess(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain > best_gain:
                best_gain, best_question = gain, question
    
    return best_gain, best_question

In [14]:
best_gain, best_question = find_best_split(training_data)
best_question

Is humidity >= 57.4?

In [15]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [16]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [17]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [18]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict",node.predictions)
        return
    
    print(spacing + str(node.question))
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [19]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is humidity >= 57.4?
--> True:
  Is humidity >= 84.4?
  --> True:
    Predict {'Klassrum': 3}
  --> False:
    Is temperature >= 19.8?
    --> True:
      Is humidity >= 81.4?
      --> True:
        Predict {'Lärarrum': 1}
      --> False:
        Predict {'Klassrum': 1}
    --> False:
      Predict {'Lärarrum': 2}
--> False:
  Predict {'Kylrum': 4}


In [20]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [21]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [22]:
testing_data = [
    [30, 15.6, 'Kylrum'],
    [50, 16, 'Klassrum'],
    [89, 21, 'Klassrum'],
    [81.7, 19,'Lärarrum'],
    [87, 19.8, 'Lärarrum'],
]

In [23]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Kylrum. Predicted: {'Kylrum': '100%'}
Actual: Klassrum. Predicted: {'Kylrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Lärarrum. Predicted: {'Lärarrum': '100%'}
Actual: Lärarrum. Predicted: {'Klassrum': '100%'}
